In [1]:
!pip install datamol
!pip install rdkit-pypi
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 60.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Project/ALVS")

Mounted at /content/drive


In [3]:
import pandas as pd
from rdkit import Chem
import glob
import datamol as dm
from sklearn.model_selection import train_test_split

In [4]:
def load_score(mol_files):
    name_scores = []
    for mol_file in mol_files:
        mols = Chem.SDMolSupplier(mol_file, sanitize=False)
        for m in mols:
            score = m.GetProp("minimizedAffinity")
            name = m.GetProp("_Name")
            try:
                smi = Chem.MolToSmiles(m)
            except:
                print(m)
            smi = smi.replace("/", "").replace("@", "").replace("\\", "")
            name_scores.append([name, smi, float(score)])
    return name_scores

In [5]:
def ensemble_merge(df):
    name_miniscore = []
    for name, dfg in df.groupby("name"):
        mini_score = dfg["score"].min()
        smis = dfg["smiles"].unique()
        # if len(smis) != 1:
        #     print(name)
        #     raise RuntimeError("smiles is not unique!")
        smi = smis[0]
        name_miniscore.append([name, smi, mini_score])
    df_merge = pd.DataFrame(name_miniscore)
    df_merge.columns = ["name", "smiles", "score"]
    return df_merge

In [6]:
def fix_smiles(smi):
    mol = dm.to_mol(smi, ordered=True, sanitize=False)
    mol = dm.fix_mol(mol)
    mol = dm.sanitize_mol(mol, sanifix=True, charge_neutral=False)
    if not mol:
        return "error"
    mol = dm.standardize_mol(mol, disconnect_metals=False, normalize=True, reionize=False, uncharge=False)
    smi = dm.standardize_smiles(dm.to_smiles(mol))
    return smi

In [7]:
mol_files = glob.glob("data/ada17/*.sdf")

In [8]:
name_scores = load_score(mol_files)

[17:27:28] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****

[17:27:28] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[17:27:36] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****

[17:27:36] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****

[17:27:36] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****

[17:27:36] 

****
Invariant Violation
inconsistent state
Violation 

[17:27:39] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[17:28:21] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



[17:28:45] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****

[17:28:45] 

****
Invariant Violation
inconsistent state
Violation occurred on line 310 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Canon.cpp
Failed Expression: ((firstFromAtom2->getBeginAtomIdx() == atom2->getIdx()) ^ (secondFromAtom2->getBeginAtomIdx() == atom2->getIdx()))
****



In [9]:
df = pd.DataFrame(name_scores)
df.columns = ["name", "smiles", "score"]

In [10]:
df_merge = ensemble_merge(df)

In [11]:
len(df_merge)

34133

In [12]:
df_merge.head()

,name,smiles,score
0,CHEMBL100081,[H]ON([H])C(=O)C1=C(N(CC2=CC=C(OCCN3([H])CCCCC...,-9.342174
1,CHEMBL100146,[H]ON([H])C(=O)[CH]1CN(C(=O)CCC2=CC=CC=C2)C2=C...,-9.059044
2,CHEMBL100280,[H]ON([H])C(=O)C1=CC=CC(CN2CCOCC2)=C1N(C)S(=O)...,-7.900249
3,CHEMBL100564,[H]ON([H])C(=O)C1=C(N(C)S(=O)(=O)C2=CC=C(OC)C=...,-8.252605
4,CHEMBL100777,[H]ON([H])C(=O)C1=CC(C2=CC3=CC=CC=C3C=C2)=CC(C...,-9.621076


In [13]:
df_merge["smiles"] = df_merge["smiles"].map(lambda x: fix_smiles(x))

Streaming output truncated to the last 5000 lines.
[17:32:16] WARNING: not removing hydrogen atom without neighbors
[17:32:16] Initializing Normalizer
[17:32:16] Running Normalizer
[17:32:16] Initializing MetalDisconnector
[17:32:16] Running MetalDisconnector
[17:32:16] Initializing Normalizer
[17:32:16] Running Normalizer
[17:32:16] WARNING: not removing hydrogen atom without neighbors
[17:32:16] Initializing Normalizer
[17:32:16] Running Normalizer
[17:32:16] Initializing MetalDisconnector
[17:32:16] Running MetalDisconnector
[17:32:16] Initializing Normalizer
[17:32:16] Running Normalizer
[17:32:16] WARNING: not removing hydrogen atom without neighbors
[17:32:16] Initializing Normalizer
[17:32:16] Running Normalizer
[17:32:16] Initializing MetalDisconnector
[17:32:16] Running MetalDisconnector
[17:32:16] Initializing Normalizer
[17:32:16] Running Normalizer
[17:32:16] WARNING: not removing hydrogen atom without neighbors
[17:32:16] Initializing Normalizer
[17:32:16] Running Normaliz

In [14]:
df_merge.head()

,name,smiles,score
0,CHEMBL100081,COc1ccc(S(=O)(=O)N(Cc2ccc(OCCN3CCCCC3)cc2)c2c(...,-9.342174
1,CHEMBL100146,COc1ccc(S(=O)(=O)N2Cc3ccccc3N(C(=O)CCc3ccccc3)...,-9.059044
2,CHEMBL100280,COc1ccc(S(=O)(=O)N(C)c2c(CN3CCOCC3)cccc2C(=O)N...,-7.900249
3,CHEMBL100564,COC(=O)C1CCCN1Cc1cccc(C(=O)NO)c1N(C)S(=O)(=O)c...,-8.252605
4,CHEMBL100777,COc1ccc(S(=O)(=O)N(C)c2c(CN3CCN(C)CC3)cc(-c3cc...,-9.621076


In [15]:
df_train, df_test = train_test_split(df_merge, test_size=0.4)

In [16]:
df_train.to_csv("preprocess_data/ada17/ada17_train.csv", index=None, sep="\t")
df_test.to_csv("preprocess_data/ada17/ada17_test.csv", index=None, sep="\t")

In [17]:
df_train.head()

,name,smiles,score
15419,ZINC20275063,COc1ccccc1NS(=O)(=O)c1ccc(NS(=O)(=O)c2ccc(F)c(...,-9.841806
26393,ZINC40564537,CCN(CC(C)C)C(=O)N(CC(=O)O)C1CCS(=O)(=O)C1,-6.570695
31570,ZINC63289617,C=CCN1C(SCC(=O)Nc2cccc([N+](=O)[O-])c2)=NNC1Cc...,-8.563109
12770,ZINC14904646,CCOc1cc(CC2=[SH]C(=O)NC2=N)ccc1OC(=O)CC1=[SH]C...,-8.282689
4611,ZINC06979201,CCCn1c(SCC(=O)N2C(C)CCCC2C)nc2cc(S(N)(=O)=O)ccc21,-7.901283


In [18]:
len(df_train)

20479

In [19]:
len(df_test)

13654

In [20]:
df_merge.to_csv("preprocess_data/ada17/ada17_processed.csv", index=None, sep="\t")